In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
wx = pd.read_csv('../assets/weather.csv')

In [3]:
wx.shape

(2944, 22)

In [4]:
wx.isnull().sum()

Station        0
Date           0
Tmax           0
Tmin           0
Tavg           0
Depart         0
DewPoint       0
WetBulb        0
Heat           0
Cool           0
Sunrise        0
Sunset         0
CodeSum        0
Depth          0
Water1         0
SnowFall       0
PrecipTotal    0
StnPressure    0
SeaLevel       0
ResultSpeed    0
ResultDir      0
AvgSpeed       0
dtype: int64

In [5]:
pd.set_option('display.max_columns', 30)
wx.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,0448,1849,,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,-,-,,M,M,M,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,0447,1850,BR,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,M,42,47,13,0,-,-,BR HZ,M,M,M,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,0446,1851,,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9


In [6]:
wx[wx['Water1'] != 'M'] # Notice there is no recordings for Water1, will drop it

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed


In [7]:
wx[wx['Depth'] != 'M'].head() # Apparently Depth is either 0 or 'M' (no recordings), will drop it also

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,0448,1849,,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
2,1,2007-05-02,59,42,51,-3,42,47,14,0,0447,1850,BR,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,0446,1851,,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9
6,1,2007-05-04,66,49,58,4,41,50,7,0,0444,1852,RA,0,M,0.0,T,29.31,30.05,10.4,8,10.8
8,1,2007-05-05,66,53,60,5,38,49,5,0,0443,1853,,0,M,0.0,T,29.40,30.10,11.7,7,12.0


In [8]:
wx.drop(['Water1', 'Depth'], axis=1, inplace=True)

In [9]:
wx.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,CodeSum,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,0448,1849,,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,-,-,,M,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,0447,1850,BR,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,M,42,47,13,0,-,-,BR HZ,M,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,0446,1851,,0.0,0.00,29.39,30.12,11.7,7,11.9


In [10]:
# There is a 'T' value for SnowFall which means trace amounts, we will approximate this to 0
wx[wx['SnowFall'] != '0.0'].head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,CodeSum,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
1,2,2007-05-01,84,52,68,M,51,57,0,3,-,-,,M,0.00,29.18,29.82,2.7,25,9.6
3,2,2007-05-02,60,43,52,M,42,47,13,0,-,-,BR HZ,M,0.00,29.44,30.08,13.3,2,13.4
5,2,2007-05-03,67,48,58,M,40,50,7,0,-,-,HZ,M,0.00,29.46,30.12,12.9,6,13.2
7,2,2007-05-04,78,51,M,M,42,50,M,M,-,-,,M,0.00,29.36,30.04,10.1,7,10.4
9,2,2007-05-05,66,54,60,M,39,50,5,0,-,-,,M,T,29.46,30.09,11.2,7,11.5


In [11]:
# There is a 'T' value for SnowFall which means trace amounts, we will approximate this to 0

wx['SnowFall'].replace('  T', '0.0', inplace=True)

In [12]:
# There is a 'T' value for PrecipTotal which means trace amounts, we will approximate this to 0

wx['PrecipTotal'].replace('  T', '0.0', inplace=True)

In [13]:
# We shall write code to make the values which are not recorded in Station 2 take the values from Station 1

wx.replace('M', np.nan, inplace=True)
wx.fillna(method='ffill', inplace=True)

In [14]:
wx.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,CodeSum,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,0448,1849,,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,14,51,57,0,3,-,-,,0.0,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,0447,1850,BR,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,-3,42,47,13,0,-,-,BR HZ,0.0,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,0446,1851,,0.0,0.00,29.39,30.12,11.7,7,11.9


In [15]:
wx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2944 entries, 0 to 2943
Data columns (total 20 columns):
Station        2944 non-null int64
Date           2944 non-null object
Tmax           2944 non-null int64
Tmin           2944 non-null int64
Tavg           2944 non-null object
Depart         2944 non-null object
DewPoint       2944 non-null int64
WetBulb        2944 non-null object
Heat           2944 non-null object
Cool           2944 non-null object
Sunrise        2944 non-null object
Sunset         2944 non-null object
CodeSum        2944 non-null object
SnowFall       2944 non-null object
PrecipTotal    2944 non-null object
StnPressure    2944 non-null object
SeaLevel       2944 non-null object
ResultSpeed    2944 non-null float64
ResultDir      2944 non-null int64
AvgSpeed       2944 non-null object
dtypes: float64(1), int64(5), object(14)
memory usage: 460.1+ KB


Dtypes to change:
1. Date - Datetime
2. Tavg - int/float
3. Depart - int/float
4. Wetbulb - int/float
5. Heat - int/float
6. Cool - int/float
7. Sunset - Datetime
8. Sunrise - Date
9. Snowfall - int/float
10. PrecipTotal - int/float
11. StnPressure - int/float
12. SeaLevel - int/float
13. AvgSpeed - int/float

In [16]:
wx['Date'] = pd.to_datetime(wx['Date'])
wx['Tavg'] = wx['Tavg'].astype(float)
wx['Depart'] = wx['Depart'].astype(float)
wx['WetBulb'] = wx['WetBulb'].astype(float)
wx['Heat'] = wx['Heat'].astype(float)
wx['Cool'] = wx['Cool'].astype(float)
wx['SnowFall'] = wx['SnowFall'].astype(float)
wx['PrecipTotal'] = wx['PrecipTotal'].astype(float)
wx['StnPressure'] = wx['StnPressure'].astype(float)
wx['SeaLevel'] = wx['SeaLevel'].astype(float)
wx['AvgSpeed'] = wx['AvgSpeed'].astype(float)

In [17]:
wx.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,CodeSum,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67.0,14.0,51,56.0,0.0,2.0,0448,1849,,0.0,0.0,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68.0,14.0,51,57.0,0.0,3.0,-,-,,0.0,0.0,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51.0,-3.0,42,47.0,14.0,0.0,0447,1850,BR,0.0,0.0,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52.0,-3.0,42,47.0,13.0,0.0,-,-,BR HZ,0.0,0.0,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56.0,2.0,40,48.0,9.0,0.0,0446,1851,,0.0,0.0,29.39,30.12,11.7,7,11.9


In [18]:
wx.dtypes

Station                 int64
Date           datetime64[ns]
Tmax                    int64
Tmin                    int64
Tavg                  float64
Depart                float64
DewPoint                int64
WetBulb               float64
Heat                  float64
Cool                  float64
Sunrise                object
Sunset                 object
CodeSum                object
SnowFall              float64
PrecipTotal           float64
StnPressure           float64
SeaLevel              float64
ResultSpeed           float64
ResultDir               int64
AvgSpeed              float64
dtype: object

In [19]:
# Exporting cleaned weather dataset

wx.to_csv('../datasets/cleaned_wx.csv', index=False)

Intuitively it does not make sense to take daily weather readings for the model (daily changes in weather temperature, etc should not have an immediate impact on the number of mosquitoes and hence the presence of the WnV). As such we will be creating a rolling average of approx 30 days for each numerical feature of each station.

In [20]:
wx.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,CodeSum,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67.0,14.0,51,56.0,0.0,2.0,0448,1849,,0.0,0.0,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68.0,14.0,51,57.0,0.0,3.0,-,-,,0.0,0.0,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51.0,-3.0,42,47.0,14.0,0.0,0447,1850,BR,0.0,0.0,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52.0,-3.0,42,47.0,13.0,0.0,-,-,BR HZ,0.0,0.0,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56.0,2.0,40,48.0,9.0,0.0,0446,1851,,0.0,0.0,29.39,30.12,11.7,7,11.9


In [21]:
# Features to apply rolling average

ft1 = wx[wx['Station'] == 1]
ft2 = wx[wx['Station'] == 2]
ft1.drop(['Sunrise', 'Sunset', 'CodeSum', 'ResultDir'], axis=1, inplace=True)
ft2.drop(['Sunrise', 'Sunset', 'CodeSum', 'ResultDir'], axis=1, inplace=True)
ft1 = ft1.groupby('Date').mean()
ft2 = ft2.groupby('Date').mean()

# Rolling
ft1 = ft1.rolling(30).mean()
ft2 = ft2.rolling(30).mean()

# The train set starts on 2007-05-29 so technically we won't need the earlier values.
# For the moving forward with less effort, we will use backfill for the dates before the first calculated value
ft1 = ft1.fillna(method='bfill')
ft2 = ft2.fillna(method='bfill')

C:\Users\cheng\Anaconda3\lib\site-packages\pandas\core\frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [22]:
ft1.head(3)

,Station,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,AvgSpeed
Date,,,,,,,,,,,,,,,
2007-05-01,1.0,75.433333,51.6,63.733333,5.166667,45.266667,54.033333,3.866667,2.6,0.0,0.059667,29.361333,30.085,8.646667,10.363333
2007-05-02,1.0,75.433333,51.6,63.733333,5.166667,45.266667,54.033333,3.866667,2.6,0.0,0.059667,29.361333,30.085,8.646667,10.363333
2007-05-03,1.0,75.433333,51.6,63.733333,5.166667,45.266667,54.033333,3.866667,2.6,0.0,0.059667,29.361333,30.085,8.646667,10.363333


In [23]:
ft2.head()

,Station,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,AvgSpeed
Date,,,,,,,,,,,,,,,
2007-05-01,2.0,75.866667,53.566667,64.733333,5.166667,45.3,54.4,3.533333,3.266667,0.0,0.041667,29.425,30.079,8.44,9.993333
2007-05-02,2.0,75.866667,53.566667,64.733333,5.166667,45.3,54.4,3.533333,3.266667,0.0,0.041667,29.425,30.079,8.44,9.993333
2007-05-03,2.0,75.866667,53.566667,64.733333,5.166667,45.3,54.4,3.533333,3.266667,0.0,0.041667,29.425,30.079,8.44,9.993333
2007-05-04,2.0,75.866667,53.566667,64.733333,5.166667,45.3,54.4,3.533333,3.266667,0.0,0.041667,29.425,30.079,8.44,9.993333
2007-05-05,2.0,75.866667,53.566667,64.733333,5.166667,45.3,54.4,3.533333,3.266667,0.0,0.041667,29.425,30.079,8.44,9.993333


In [24]:
ft1.shape

(1472, 15)

In [25]:
ft2.shape

(1472, 15)

In [26]:
ft_ovr = pd.concat([ft1,ft2], axis=0)
ft_ovr.shape

(2944, 15)

In [27]:
#wx = wx.merge(ft_ovr, how='left', left_on='Date', right_on='Date')
#wx.head(1)

In [28]:
wx = wx.merge(ft_ovr, how='left', left_on=['Date', 'Station'], right_on=['Date', 'Station'])

In [29]:
wx.head(2)

,Station,Date,Tmax_x,Tmin_x,Tavg_x,Depart_x,DewPoint_x,WetBulb_x,Heat_x,Cool_x,Sunrise,Sunset,CodeSum,SnowFall_x,PrecipTotal_x,...,AvgSpeed_x,Tmax_y,Tmin_y,Tavg_y,Depart_y,DewPoint_y,WetBulb_y,Heat_y,Cool_y,SnowFall_y,PrecipTotal_y,StnPressure_y,SeaLevel_y,ResultSpeed_y,AvgSpeed_y
0,1,2007-05-01,83,50,67.0,14.0,51,56.0,0.0,2.0,0448,1849,,0.0,0.0,...,9.2,75.433333,51.600000,63.733333,5.166667,45.266667,54.033333,3.866667,2.600000,0.0,0.059667,29.361333,30.085,8.646667,10.363333
1,2,2007-05-01,84,52,68.0,14.0,51,57.0,0.0,3.0,-,-,,0.0,0.0,...,9.6,75.866667,53.566667,64.733333,5.166667,45.300000,54.400000,3.533333,3.266667,0.0,0.041667,29.425000,30.079,8.440000,9.993333


In [30]:
wx.columns

Index(['Station', 'Date', 'Tmax_x', 'Tmin_x', 'Tavg_x', 'Depart_x',
       'DewPoint_x', 'WetBulb_x', 'Heat_x', 'Cool_x', 'Sunrise', 'Sunset',
       'CodeSum', 'SnowFall_x', 'PrecipTotal_x', 'StnPressure_x', 'SeaLevel_x',
       'ResultSpeed_x', 'ResultDir', 'AvgSpeed_x', 'Tmax_y', 'Tmin_y',
       'Tavg_y', 'Depart_y', 'DewPoint_y', 'WetBulb_y', 'Heat_y', 'Cool_y',
       'SnowFall_y', 'PrecipTotal_y', 'StnPressure_y', 'SeaLevel_y',
       'ResultSpeed_y', 'AvgSpeed_y'],
      dtype='object')

In [31]:
wx.drop(['Tmax_x', 'Tmin_x', 'Tavg_x', 'Depart_x',
       'DewPoint_x', 'WetBulb_x', 'Heat_x', 'Cool_x',
       'SnowFall_x', 'PrecipTotal_x', 'StnPressure_x', 'SeaLevel_x',
       'ResultSpeed_x', 'AvgSpeed_x'], axis=1, inplace=True)

In [32]:
wx.columns

Index(['Station', 'Date', 'Sunrise', 'Sunset', 'CodeSum', 'ResultDir',
       'Tmax_y', 'Tmin_y', 'Tavg_y', 'Depart_y', 'DewPoint_y', 'WetBulb_y',
       'Heat_y', 'Cool_y', 'SnowFall_y', 'PrecipTotal_y', 'StnPressure_y',
       'SeaLevel_y', 'ResultSpeed_y', 'AvgSpeed_y'],
      dtype='object')

In [33]:
wx.columns = ['Station', 'Date', 'Sunrise', 'Sunset', 'CodeSum', 'ResultDir',
       'Tmax', 'Tmin', 'Tavg', 'Depart', 'DewPoint', 'WetBulb',
       'Heat', 'Cool', 'SnowFall', 'PrecipTotal', 'StnPressure',
       'SeaLevel', 'ResultSpeed', 'AvgSpeed']

wx.head()

,Station,Date,Sunrise,Sunset,CodeSum,ResultDir,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,AvgSpeed
0,1,2007-05-01,0448,1849,,27,75.433333,51.600000,63.733333,5.166667,45.266667,54.033333,3.866667,2.600000,0.0,0.059667,29.361333,30.085,8.646667,10.363333
1,2,2007-05-01,-,-,,25,75.866667,53.566667,64.733333,5.166667,45.300000,54.400000,3.533333,3.266667,0.0,0.041667,29.425000,30.079,8.440000,9.993333
2,1,2007-05-02,0447,1850,BR,4,75.433333,51.600000,63.733333,5.166667,45.266667,54.033333,3.866667,2.600000,0.0,0.059667,29.361333,30.085,8.646667,10.363333
3,2,2007-05-02,-,-,BR HZ,2,75.866667,53.566667,64.733333,5.166667,45.300000,54.400000,3.533333,3.266667,0.0,0.041667,29.425000,30.079,8.440000,9.993333
4,1,2007-05-03,0446,1851,,7,75.433333,51.600000,63.733333,5.166667,45.266667,54.033333,3.866667,2.600000,0.0,0.059667,29.361333,30.085,8.646667,10.363333


In [34]:
wx.shape

(2944, 20)

In [35]:
# Exporting cleaned weather dataset

wx.to_csv('../datasets/cleaned_roll_wx.csv', index=False)